In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# ConversationSummaryMemory
# llm을 사용 == 메모리를 실행하는 데 비용이 든다는 뜻
# 그대로 대화를 저장하는 것이 아니라 요약한 대화를 저장
# 초기 비용은 크지만, 데이터가 커질수록(대화가 길어질수록) 비용을 줄일 수 있음
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

In [7]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

In [8]:
add_message("Hi I'm Nicloas, I live in South Korea", "Wow that is so cool!")

In [9]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [10]:
get_history()

{'history': 'The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human then mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'}